# Computational Bionics: Project Exercise 1

Group members:
- Yash Waware
- Julian Lork
- Florian Frech (12308544)

## Task 1: Modeling the Leg

### 1.1 Description

The following section consider the model of the leg, described by an ODE model.

A test person with a body mass of $m_b = 100$ kg is chosen.

Following state variables are involved:
- $q_1$: Extension angle of the **hip** around then transversal axis ($xy$-plane)
- $\omega_1$: Angular velocity of the tight
- $q_2$: Rotation angle of the **knee** around the transversal axis ($xy$-plane) 
- $\omega_2$ Angular velocity of the shank

The segement masses are considered as point masses in the middle of each segment.

![](LegModel.png)

|Variable|Description|
|--------|-----------|
|$r_H$|Position vector of the hip to the reference system|
|$H$|Hip Joint|
|$K$|Knee Joint|
|$O$|Tight|
|$U$|Shank|
|$S$|Centre of Mass of the Foot|
|$SO$|Centre of Mass of the Tight|
|$SU$|Centre of Mass Shank|
|$m_1$|Tight Mass|
|$m_2$|Shank Mass|
|$m_3$|Foot Point Mass|
|$J_1$|Moment of Inertia of the Tight|
|$J_2$|Moment of Inertia of the Shank|
|$\alpha_H$|Flexion / Extension Angle of the Hip|
|$\alpha_K$|Flexion / Extension Angle of the Knee|


Following relations for the partial masses of the legs are assumed:

|Segment| Mass m [kg]|
|------|--------------|
|Tight|  $m_1 = 0.090 \cdot m_b + 0.73 = 9.73$|
|Shank| $m_2 = 0.055 \cdot m_b - 0.43 = 5.07$|
|Foot| $m_3 = 0.001 \cdot m_b + 0.34 = 0.44$|

### 1.2 Simplified Box Model

![](BoxModelLegModel.png)

**Input:**
- anatomical data
- segment weights
- proportions of the limbs
- length and position of the segments
- gait data for one cycle
- Flexion / Extension **angles** of the ankle, knee, and hip joint

**Equations:**

**Output:**
- Ground reaction **forces** in y- and x- direction
- **Moments** around foot, knee, and hip joints


### 1.3 Setting
- Length of the Tight: $l_O = 0.410$ m
- Length of the Shank: $l_U = 0.415$ m

https://personal.cityu.edu.hk/meachan/Online%20Anthropometry/Chapter2/Ch2-5.htm

https://pmc.ncbi.nlm.nih.gov/articles/PMC5305206/

### 1.4 Derivation 

**Relation between common measured joint angle and the chosen generalized coordinates**

$$q_1 = \alpha_H + \frac{3}{2}\pi$$

$$q_2 = q_1 - \alpha_K = \alpha_H + \frac{3}{2}\pi - \alpha_K$$

**Position vectors** to the centre of mass of tight $SO$, shank $SU$, and foot $S$ with respect to the hip $H$

https://scienceworld.wolfram.com/physics/DoublePendulum.html

https://en.wikipedia.org/wiki/Double_pendulum


$$
\mathbf{r}_{SO} = 0.5 \cdot l_O \cdot \begin{bmatrix} 
                   \cos q_1 \\
                   \sin q_1 \\
                   0 \end{bmatrix} \quad \quad

\mathbf{r}_{SU} = \begin{bmatrix}
                   l_O \cos q_1 + 0.5 \cdot l_U \cos q_2 \\
                   l_O \sin q_1 + 0.5 \cdot l_U \sin q_2 \\
                   0 \end{bmatrix} \quad \quad

\mathbf{r}_{S} = \begin{bmatrix}
                   l_O \cos q_1 + l_U \cos q_2 \\
                   l_O \sin q_1 + l_U \sin q_2 \\
                   0 \end{bmatrix} \quad \quad                   
$$

![](SketchTask1.png)

**Velocity vectors** to the centre of mass of tight $SO$, shank $SU$, and foot $S$

$$
\mathbf{v}_{SO} = 0.5 \cdot l_O \cdot \dot{q}_1\begin{bmatrix} - \sin q_1 \\ \cos q_1 \\ 0 \end{bmatrix}
$$

$$
\mathbf{v}_{SU} = l_O \cdot \dot{q}_1 \begin{bmatrix}  - \sin q_1 \\ \cos q_1 \\ 0 \end{bmatrix} + 0.5 \cdot l_U \cdot \dot{q}_2 \begin{bmatrix}  - \sin q_2 \\ \cos q_2 \\ 0 \end{bmatrix}
$$

$$
\mathbf{v}_{S} = l_O \cdot \dot{q}_1 \begin{bmatrix}  - \sin q_1 \\ \cos q_1 \\ 0 \end{bmatrix} + l_U \cdot \dot{q}_2 \begin{bmatrix}  - \sin q_2 \\ \cos q_2 \\ 0 \end{bmatrix}
$$

In [ ]:
# define numerical values
l_O_val = 0.410 # length of the tigh [m]
l_U_val = 0.415 # length of the shank [m]

mO_val = 9.73 # mass of the thigh [kg]
mU_val = 5.07 # mass of the shank [kg]
mF_val = 0.44 # mass of the foot [kg]

g_val = 9.81 # gravity [m/s^2]

In [ ]:
import sympy as sp

# define symbolic variables
l_O, l_U, q1, q2, w1, w2, t = sp.symbols('l_O l_U q1 q2 w_1  w_2 t')
mO, mU, mF = sp.symbols('m_O m_U m_F')
g = sp.symbols('g')

q1 = sp.Function('q1')(t)
q2 = sp.Function('q2')(t)

w1 = sp.Function('w_1')(t)
w2 = sp.Function('w_2')(t)

# Position vectors
r_SO = sp.Matrix([0.5 * l_O * sp.cos(q1), 0.5 * l_O *sp.sin(q1), 0])
r_SU = sp.Matrix([l_O * sp.cos(q1) + 0.5 * l_U * sp.cos(q2), l_O * sp.sin(q1) + 0.5 * l_U * sp.sin(q2), 0])
r_S = sp.Matrix([l_O * sp.cos(q1) + l_U * sp.cos(q2), l_O * sp.sin(q1) + l_U * sp.sin(q2), 0])

# Define velocity vectors
v_SO = 0.5 * l_O * w1 * sp.Matrix([[-sp.sin(q1)], [sp.cos(q1)], [0]])
v_SU = l_O * w1 * sp.Matrix([[-sp.sin(q1)], [sp.cos(q1)], [0]]) + 0.5 * l_U * w2 * sp.Matrix([[-sp.sin(q2)], [sp.cos(q2)], [0]])
v_S = l_O * w1 * sp.Matrix([[-sp.sin(q1)], [sp.cos(q1)], [0]]) + l_U * w2 * sp.Matrix([[-sp.sin(q2)], [sp.cos(q2)], [0]])

#v_SO = sp.diff(r_SO, t)
#v_SU = sp.diff(r_SU, t)
#v_S = sp.diff(r_S, t)

**Kinetic Energy T**

$$
T_1 = \frac{1}{2} m_O \mathbf{v}_{SO}^T \mathbf{v}_{SO}
$$

$$
T_2 = \frac{1}{2} m_U \mathbf{v}_{SU}^T \mathbf{v}_{SU}
$$

$$
T_3 = \frac{1}{2} m_F \mathbf{v}_{S}^T \mathbf{v}_{S}
$$

In [2]:
# Kinetic energy
T1 = 0.5 * mO * v_SO.dot(v_SO)
T2 = 0.5 * mU * v_SU.dot(v_SU)
T3 = 0.5 * mF * v_S.dot(v_S)
T = T1 + T2 + T3
T = T.simplify()

T

0.125*l_O**2*m_O*w_1(t)**2 + 0.5*m_F*(l_O**2*w_1(t)**2 + 2*l_O*l_U*w_1(t)*w_2(t)*cos(q1(t) - q2(t)) + l_U**2*w_2(t)**2) + 0.5*m_U*(1.0*l_O**2*w_1(t)**2 + 1.0*l_O*l_U*w_1(t)*w_2(t)*cos(q1(t) - q2(t)) + 0.25*l_U**2*w_2(t)**2)

**Potential Energy V**

$$
V_1 = m_O \cdot g \cdot 0.5 \cdot l_O \cdot \sin q_1
$$

$$
V_2 = m_U \cdot g \cdot ( l_O \sin q_1 + 0.5 \cdot l_U \sin q_2)
$$

$$
V_3 = m_F \cdot g \cdot ( l_O \sin q_1 + \cdot l_U \sin q_2)
$$

In [3]:
# Potential energy
V1 = mO * g * r_SO[1]
V2 = mU * g * r_SU[1]
V3 = mF * g * r_S[1]
V = V1 + V2 + V3
V = V.simplify()

V

g*(0.5*l_O*m_O*sin(q1(t)) + m_F*(l_O*sin(q1(t)) + l_U*sin(q2(t))) + m_U*(l_O*sin(q1(t)) + 0.5*l_U*sin(q2(t))))

**Lagrange Function**

$$L = T - V$$

$$
L = T_1 + T_2 + T_3 - V_1 - V_2 - V_3
$$

$$
L = \frac{1}{2} \left[ m_O \mathbf{v}_{SO}^T \mathbf{v}_{SO} + m_U \mathbf{v}_{SU}^T \mathbf{v}_{SU} + m_F \mathbf{v}_{S}^T \mathbf{v}_{S} \right]
- g \cdot \left[ m_O \cdot 0.5 \cdot l_O \cdot \sin q_1 +  m_U \cdot ( l_O \sin q_1 + 0.5 \cdot l_U \sin q_2) + m_F \cdot ( l_O \sin q_1 + \cdot l_U \sin q_2) \right]
$$


In [4]:
# Lagrangian
L = T - V

# Derivatives: dL / dw_i
dLdw1 = sp.diff(L, w1).simplify()
dLdw2 = sp.diff(L, w2).simplify()

# Derivatives: d/dt (dL / dw_i)
ddt_dLdw1 = sp.diff(dLdw1, t)
ddt_dLdw2 = sp.diff(dLdw2, t)

# Derivatives: dL / dq_i
dLdq1 = sp.diff(L, q1)
dLdq2 = sp.diff(L, q2)

In [5]:
M_H, M_K, w1_dot, w2_dot  = sp.symbols(r'M_H M_K \dot{w}_1 \dot{w}_2')

# Equations of motion for q1 and q2
eq1 = ddt_dLdw1 - dLdq1 - M_H
eq2 = ddt_dLdw2 - dLdq2 - M_K

eq1 = eq1.subs(w1.diff(t), w1_dot).subs(w2.diff(t), w2_dot)
eq2 = eq2.subs(w2.diff(t), w2_dot).subs(w1.diff(t), w1_dot)

eq1 = eq1.simplify()
eq2 = eq2.simplify()

w1_dot = sp.solve(eq1, w1_dot, dict=True, simplify=True, rational=True)
w2_dot = sp.solve(eq2, w2_dot, dict=True, simplify=True, rational=True)

w1_dot, w2_dot

([{\dot{w}_1: 2*(2*M_H - 2*\dot{w}_2*l_O*l_U*m_F*cos(q1(t) - q2(t)) - \dot{w}_2*l_O*l_U*m_U*cos(q1(t) - q2(t)) - 2*g*l_O*m_F*cos(q1(t)) - g*l_O*m_O*cos(q1(t)) - 2*g*l_O*m_U*cos(q1(t)) - 2*l_O*l_U*m_F*w_1(t)*w_2(t)*sin(q1(t) - q2(t)) + 2*l_O*l_U*m_F*w_2(t)*sin(q1(t) - q2(t))*Derivative(q1(t), t) - 2*l_O*l_U*m_F*w_2(t)*sin(q1(t) - q2(t))*Derivative(q2(t), t) - l_O*l_U*m_U*w_1(t)*w_2(t)*sin(q1(t) - q2(t)) + l_O*l_U*m_U*w_2(t)*sin(q1(t) - q2(t))*Derivative(q1(t), t) - l_O*l_U*m_U*w_2(t)*sin(q1(t) - q2(t))*Derivative(q2(t), t))/(l_O**2*(4*m_F + m_O + 4*m_U))}],
 [{\dot{w}_2: 2*(2*M_K - 2*\dot{w}_1*l_O*l_U*m_F*cos(q1(t) - q2(t)) - \dot{w}_1*l_O*l_U*m_U*cos(q1(t) - q2(t)) - 2*g*l_U*m_F*cos(q2(t)) - g*l_U*m_U*cos(q2(t)) + 2*l_O*l_U*m_F*w_1(t)*w_2(t)*sin(q1(t) - q2(t)) + 2*l_O*l_U*m_F*w_1(t)*sin(q1(t) - q2(t))*Derivative(q1(t), t) - 2*l_O*l_U*m_F*w_1(t)*sin(q1(t) - q2(t))*Derivative(q2(t), t) + l_O*l_U*m_U*w_1(t)*w_2(t)*sin(q1(t) - q2(t)) + l_O*l_U*m_U*w_1(t)*sin(q1(t) - q2(t))*Derivative(q1(t)

**Approach:**
$$\frac{\partial}{\partial t} \left(\frac{\partial L}{\partial \dot{q}_i} - \frac{\partial L}{\partial q_i} \right) = Q_i$$

$$
\frac{\partial}{\partial t} \left(\frac{\partial L}{\partial \dot{q}_i} \right) = 

\frac{\partial}{\partial t} \left[ \frac{1}{2} \left[ m_O \mathbf{v}_{SO}^T \mathbf{v}_{SO} + m_U \mathbf{v}_{SU}^T \mathbf{v}_{SU} + m_F \mathbf{v}_{S}^T \mathbf{v}_{S} \right] \right]
$$

$$
\frac{\partial}{\partial t} \left(\frac{\partial L}{\partial \omega_1} \right) =
$$

$$
\frac{\partial}{\partial t} \left(\frac{\partial L}{\partial \omega_2} \right) =
$$

$$
\frac{\partial L}{\partial q_1} = 
$$

$$
\frac{\partial L}{\partial q_2} = 
$$

----------------------------------------------------------

In [1]:
import numpy as np
import pandas as pd

In [25]:
# read gait file
filename = 'gait_data.xls'

gait_data = pd.read_excel(filename, engine='xlrd')

gait_data["gait_%"]


0      0
1      1
2      2
3      3
4      4
      ..
95    95
96    96
97    97
98    98
99    99
Name: gait_%, Length: 100, dtype: int64

In [22]:
GRFz = gait_data['gait_%']
GRFx = gait_data[2]

MX_F = gait_data[3]
Flex_Ext_F = gait_data[4]

MX_K = gait_data[5]
alpha_K = gait_data[6]

MX_H = gait_data[7]
alpha_H = gait_data[8]

alpha_H

KeyError: 'gait_%'

In [12]:
# Transform angles
q1_gait = np.deg2rad(alpha_H) + 3/2 * np.pi
q2_gait = q1_gait - np.deg2rad(alpha_K)

TypeError: loop of ufunc does not support argument 0 of type str which has no callable deg2rad method

In [ ]:
# Time step
dt = 1
t_eval = np.arange(0, 101, dt)

# Initial conditions
q1_0 = 
q2_0 = 0.01
w1_0 = 0
w2_0 = 0

In [ ]:
# Function to define the ODE
def leg_model(t, y):
    q1, q2, w1, w2 = y
    
    domega1 = w1_dot[0].subs({q1: q1_0, q2: q2_0, w1: w1_0, w2: w2_0})
    domega2 = w2_dot[0].subs({q1: q1_0, q2: q2_0, w1: w1_0, w2: w2_0})

    dq1 = w1
    dq2 = w2

    return [domega1, domega2, dq1, dq2]
